# sample_text_02 - 풀이

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트
plt.rcParams['axes.unicode_minus'] = False      # 마이너스 부호


## 문제 1

01.
 
(card1을 이용하여) ’연간 평균 잔고액(BALANCE)’이 많을수록, 그리고 ‘신용카드 서비스 이용기간(TENURE)’이 길수록 ‘신용카드 한도(CREDIT_LIMIT)’ 역시 높을 것으로 예상해볼 수 있다. 이들 변수 간의 관계를 파악하여, 추후 고객의 신용카드 한도 조정에 근거 자료로 활용하고자 한다.

‘신용 카드 서비스 이용기간(TENURE)’ 별로 ‘연간 평균 잔고액(BALANCE)’과 ‘신용카드 한도(CREDIT_LIMIT)’ 간 피어슨(Pearson) 상관 계수를 계산하고, 이 중 가장 큰 값을 구하시오.
- 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지 기술하시오. (답안예시 : 0.12)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# 데이터 로드 (예시로 데이터프레임을 생성)
# 실제 데이터는 card1에서 로드해야 합니다.
data = {
    'TENURE': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'BALANCE': [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500],
    'CREDIT_LIMIT': [5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000]
}
df = pd.DataFrame(data)

# TENURE 별로 그룹화하여 BALANCE와 CREDIT_LIMIT 간의 피어슨 상관 계수를 계산
correlation_results = {}

for tenure, group in df.groupby('TENURE'):
    correlation, _ = pearsonr(group['BALANCE'], group['CREDIT_LIMIT'])
    correlation_results[tenure] = correlation

# 가장 큰 피어슨 상관 계수 찾기
max_correlation = max(correlation_results.values())
max_correlation_rounded = round(max_correlation, 2)

# 결과 출력
print("TENURE 별 피어슨 상관 계수:", correlation_results)
print("가장 큰 피어슨 상관 계수 (소수점 둘째 자리까지):", max_correlation_rounded)

## 문제 2

02.
 
(card1을 이용하여) 신용카드 판매 전략을 수립하기 위해 고객 세분화(Customer Segmentation) 를 수행할 수 있다. 일시불 구매 금액이 높은 고객 Segment를 도출하기 위해 다음 단계에 따라 분석을 수행하고 질문에 답하시오. (배점 : 10점)
단계 1: ‘고객 ID(CUST_ID)’를 제외한 모든 변수(17개)에 대해 Z-score 표준화(Standardization) 한다.
단계 2: 표준화된 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~5개 중 K-means Silhouette 를 통해 구한 최적의 K로 설정한다.
단계 3: 단계 2에서 도출한 각 군집 별로 ‘일시불 구매 총액(ONEOFF_PURCHASES)’의 평균을 계산한다.
툴별 가이드

| Brightics | Seed=1234
문제 지시 외 Default 값 사용 |
| --- | --- |
| R | library(cluster)
set.seed(12345)
표준화 : scale() 함수의 center=T, scale=T 옵션 사용
Silhouette : silhouette() 함수의 sil_width 평균값 기준
문제 지시 외 Default 값 사용 |
| Python | from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
random_state=1234
문제 지시 외 Default 값 사용 |

군집 별 ‘일시불 구매 총액(ONEOFF_PURCHASES)’의 평균 중 가장 큰 값을 구하시오.
- 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지 기술하시오. (답안예시 : 1200.34)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 데이터 로드 (예시로 'card1.csv'라는 파일을 사용)
# df = pd.read_csv('card1.csv')

# 예시 데이터 생성 (실제 데이터로 대체해야 함)
np.random.seed(1234)
data = np.random.rand(100, 17)  # 100개의 샘플과 17개의 변수
df = pd.DataFrame(data, columns=[f'feature_{i}' for i in range(1, 18)])
df['CUST_ID'] = range(1, 101)  # 고객 ID 추가
df['ONEOFF_PURCHASES'] = np.random.rand(100) * 1000  # 일시불 구매 총액

# 단계 1: 고객 ID를 제외한 모든 변수에 대해 Z-score 표준화
features = df.drop(columns=['CUST_ID', 'ONEOFF_PURCHASES'])
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# 단계 2: K-means 군집 분석 수행
silhouette_scores = []
K_range = range(2, 6)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1234)
    kmeans.fit(standardized_features)
    score = silhouette_score(standardized_features, kmeans.labels_)
    silhouette_scores.append(score)

# 최적의 K 찾기
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"최적의 K: {optimal_k}")

# 최적의 K로 K-means 재적용
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=1234)
df['Cluster'] = kmeans_optimal.fit_predict(standardized_features)

# 단계 3: 각 군집 별로 '일시불 구매 총액'의 평균 계산
cluster_means = df.groupby('Cluster')['ONEOFF_PURCHASES'].mean()
print("각 군집 별 '일시불 구매 총액'의 평균:")
print(cluster_means)

# 가장 큰 평균값 찾기
max_mean = cluster_means.max()
max_mean_rounded = round(max_mean, 2)
print(f"가장 큰 평균값 (소수점 둘째 자리까지): {max_mean_rounded}")

## 문제 3

03.
 
(card1을 이용하여) 이번에는 ‘일시불 구매 총액(ONEOFF_PURCHASES)’ 예측 모델을 Target Marketing에 활용하고자 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오. (배점 : 15점)
단계 1: ‘고객 ID(CUST_ID)’가 4의 배수가 아닌 데이터를 Train Set으로, 4의 배수인 데이터를 Test Set으로 분할한다.
단계 2: Train Set으로 아래 조건에 따라 의사결정나무 회귀모델을 학습한다.
            - 독립 변수(총 16개): ‘고객 ID(CUST_ID)’, ‘일시불 구매 총액(ONEOFF_PURCHASES)’을 제외한 모든 컬럼
            - 종속 변수: ‘일시불 구매 총액(ONEOFF_PURCHASES)’
툴별 가이드

| Brightics | Seed=1234
문제 지시 외 Default 값 사용 |
| --- | --- |
| R | set.seed(1234)
library(tree)
Decision Tree Regression : tree() 함수 사용
문제 지시 외 Default 값 사용 |
| Python | from sklearn.tree import DecisionTreeRegressor
random_state=1234
문제에서 지시한 것 외에는 Default 값 사용 |

단계 3: 생성된 모델을 Test Set에 적용하여 ‘일시불 구매 총액(ONEOFF_PURCHASES)’을 예측한다.

단계 3에서 얻은 예측 결과를 평가하기 위해, 아래 정의된 Measure B를 구하시오.

$$B=(\frac{1}{n}\sum_{i=1}^{n} (y_{i}-\hat{y_{i}})^{2})^{\frac{1}{2}}$$

 $\hat{y_{i}}$ : 예측값,  $y_{i}$ : 실제값

- B는 소수점 둘째 자리에서 반올림하여 소수점 첫째 자리까지 기술하시오. (답안예시 : 1200.3)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# 데이터 로드 (예시로 'card1.csv' 파일을 사용한다고 가정)
data = pd.read_csv('card1.csv')

# 단계 1: 고객 ID(CUST_ID)가 4의 배수가 아닌 데이터를 Train Set으로, 4의 배수인 데이터를 Test Set으로 분할
train_set = data[data['CUST_ID'] % 4 != 0]
test_set = data[data['CUST_ID'] % 4 == 0]

# 독립 변수와 종속 변수 설정
X_train = train_set.drop(columns=['CUST_ID', 'ONEOFF_PURCHASES'])
y_train = train_set['ONEOFF_PURCHASES']
X_test = test_set.drop(columns=['CUST_ID', 'ONEOFF_PURCHASES'])
y_test = test_set['ONEOFF_PURCHASES']

# 단계 2: 의사결정나무 회귀모델 학습
model = DecisionTreeRegressor(random_state=1234)
model.fit(X_train, y_train)

# 단계 3: Test Set에 모델 적용하여 예측
y_pred = model.predict(X_test)

# Measure B 계산
B = np.sqrt(np.mean((y_test - y_pred) ** 2))

# 결과 출력
print("Measure B:", round(B, 1))